In [1]:
from __future__ import division, print_function, absolute_import, unicode_literals
import numpy as np

import pygsti
from pygsti.construction import std1Q_XYI as std
from pygsti.construction import std2Q_XYICNOT as std2Q
from pygsti.extras import circuit

## Test Labels

In [2]:
print(list(map(str,std.gs_target.gates.keys())))
print(std.gs_target.gates.keys())

['Gi', 'Gx', 'Gy']
odict_keys([Label('Gi',), Label('Gx',), Label('Gy',)])


In [3]:
print(std.gs_target['Gx'])

Fully Parameterized gate with shape (4, 4)
 1.00   0   0   0
   0 1.00   0   0
   0   0   0-1.00
   0   0 1.00   0



In [4]:
std.gs_target.probs(('Gx','Gy'))

OutcomeLabelDict([(('0',), 0.5000000000000001), (('1',), 0.4999999999999999)])

## Test Circuit

In [5]:
c = pygsti.obj.Circuit(gatestring=('Gx','Gy'), num_lines=1)

In [6]:
print(c.line_items)
print(c.tup)
print(c.str)
print(c)

[[Label('Gx',), Label('Gy',)]]
(Label('Gx',), Label('Gy',))
GxGy
Qubit 0 ---Gx|-|Gy|-|--



In [7]:
from pygsti.objects import Label as L
gs = pygsti.obj.GateString( (L('Gx',0),L('Gy',1),L('Gcnot',(1,2)) ))
c2 = pygsti.obj.Circuit(gatestring=gs, num_lines=3)

In [8]:
print(c2.line_items)
print(c2.tup)
print(c2.str)
print(c2)

[[Label('Gx', 0), Label('I', 0)], [Label('Gy', 1), Label('Gcnot', 1, 2)], [Label('I', 2), Label('Gcnot', 1, 2)]]
(Label('Gx', 0), Label('Gy', 1), Label('Gcnot', 1, 2))
Gx:0Gy:1Gcnot:1:2
Qubit 0 ---|Gx |-|  |---
Qubit 1 ---|Gy |-Gcnot:1:2|-|--
Qubit 2 ---|  |-Gcnot:1:2|-|--



In [9]:
c3 = pygsti.obj.Circuit(gatestring=[ ('Gx','Q0'), ('Gy','Q1'), ('Gcnot','Q1','Q2') ], line_labels=('Q0','Q1','Q2'))
print(c3.line_items)
print(c3.tup)
print(c3.str)
print(c3)

[[Label('Gx', 'Q0'), Label('I', 'Q0')], [Label('Gy', 'Q1'), Label('Gcnot', 'Q1', 'Q2')], [Label('I', 'Q2'), Label('Gcnot', 'Q1', 'Q2')]]
(Label('Gx', 'Q0'), Label('Gy', 'Q1'), Label('Gcnot', 'Q1', 'Q2'))
Gx:Q0Gy:Q1Gcnot:Q1:Q2
Qubit 0 ---|Gx |-|  |---
Qubit 1 ---|Gy |-Gcnot:Q1:Q2|-|--
Qubit 2 ---|  |-Gcnot:Q1:Q2|-|--



In [10]:
#Test initializing an empty circuit & debug circuit -> tuple lazy eval
c0 = pygsti.obj.Circuit(num_lines=3)
c1 = pygsti.obj.Circuit(gatestring=( ('Gx',0), ('Gcnot',0,1)), num_lines=3)
print("HERE1")
print(c1[0])
c1.insert_layer([pygsti.obj.Label('Gy',1)],0)
print("HERE2")
print(c1[0])

HERE1
Gx:0
HERE2
Gy:1


### Probabilities from circuit (simulation)

In [11]:
gs1 = std.gs_target.copy()
print(gs1.probs(c))
print(c.simulate(gs1))

OutcomeLabelDict([(('0',), 0.5000000000000001), (('1',), 0.4999999999999999)])
OutcomeLabelDict([(('0',), 0.5000000000000001), (('1',), 0.4999999999999999)])


In [12]:
from pygsti.objects.gatemapcalc import GateMapCalc                                                                                                                                   
nQubits = 3
basis1Q = pygsti.obj.Basis("pp",2)
v0 = pygsti.construction.basis_build_vector("0", basis1Q)
v1 = pygsti.construction.basis_build_vector("1", basis1Q)

gs3 = pygsti.obj.GateSet()
gs3.stateSpaceLabels = ('Q0','Q1','Q2')
gs3.preps['rho0'] = pygsti.obj.TensorProdSPAMVec('prep',
    [pygsti.obj.TPParameterizedSPAMVec(v0) for i in range(nQubits)] )
gs3.povms['Mdefault'] = pygsti.obj.TensorProdPOVM( 
    [ pygsti.obj.TPPOVM([('0',v0),('1',v1)] ) for i in range(nQubits) ] )
Gx = std.gs_target['Gx'].copy()
Gy = std.gs_target['Gy'].copy()
for i in range(nQubits):
    gs3[('Gx','Q%d' % i)] = Gx
    gs3[('Gy','Q%d' % i)] = Gy

Gcnot = std2Q.gs_target['Gcnot'].copy()
gs3[('Gcnot','Q0','Q1')] = Gcnot
gs3[('Gcnot','Q1','Q2')] = Gcnot

gs3._calcClass = GateMapCalc
print(gs3.probs(c3))
print(c3.simulate(gs3))

OutcomeLabelDict([(('000',), 0.2500000000000003), (('001',), 0.2500000000000003), (('010',), 0.0), (('011',), 0.0), (('100',), 0.2500000000000002), (('101',), 0.2500000000000002), (('110',), 0.0), (('111',), 0.0)])
OutcomeLabelDict([(('000',), 0.2500000000000003), (('001',), 0.2500000000000003), (('010',), 0.0), (('011',), 0.0), (('100',), 0.2500000000000002), (('101',), 0.2500000000000002), (('110',), 0.0), (('111',), 0.0)])


In [13]:
from pygsti.objects.gatemapcalc import GateMapCalc     
from pygsti.objects.labeldicts import StateSpaceLabels
nQubits = 3
basis1Q = pygsti.obj.Basis("pp",2)
v0 = pygsti.construction.basis_build_vector("0", basis1Q)
v1 = pygsti.construction.basis_build_vector("1", basis1Q)

gs2 = pygsti.obj.GateSet()
gs2.stateSpaceLabels = (0,1,2)
gs2.preps['rho0'] = pygsti.obj.TensorProdSPAMVec('prep',
    [pygsti.obj.TPParameterizedSPAMVec(v0) for i in range(nQubits)] )
gs2.povms['Mdefault'] = pygsti.obj.TensorProdPOVM( 
    [ pygsti.obj.TPPOVM([('0',v0),('1',v1)] ) for i in range(nQubits) ] )

Gx = std.gs_target['Gx'].copy()
Gy = std.gs_target['Gy'].copy()
for i in range(nQubits):
    gs2[('Gx',i)] = Gx
    gs2[('Gy',i)] = Gy

Gcnot = std2Q.gs_target['Gcnot'].copy()
gs2[('Gcnot',0,1)] = Gcnot
gs2[('Gcnot',1,2)] = Gcnot

gs2._calcClass = GateMapCalc
print(gs2.probs(c2))
print(c2.simulate(gs2))

OutcomeLabelDict([(('000',), 0.2500000000000003), (('001',), 0.2500000000000003), (('010',), 0.0), (('011',), 0.0), (('100',), 0.2500000000000002), (('101',), 0.2500000000000002), (('110',), 0.0), (('111',), 0.0)])
OutcomeLabelDict([(('000',), 0.2500000000000003), (('001',), 0.2500000000000003), (('010',), 0.0), (('011',), 0.0), (('100',), 0.2500000000000002), (('101',), 0.2500000000000002), (('110',), 0.0), (('111',), 0.0)])


## Unitary evolution in pyGSTi

In [14]:
#Unitary gateset in pygsti
import scipy.linalg as spl
from pygsti.objects.gatematrixcalc import UnitaryGateMatrixCalc
from pygsti.objects.gatemapcalc import UnitaryGateMapCalc
X = np.array([[0.,1.],[1.,0.]])
Ut = np.array([[1.,0.],[0.,np.exp(np.pi*1j/4)]])
Ux = 1j*spl.expm( -1j* np.pi/4 * X) # pi/2
#Ux = np.array([[0.,1.],[1.,0.]]) # pi
v0 = np.array([[1],[0]],'d')
v1 = np.array([[0],[1]],'d')

gs = pygsti.obj.GateSet()
#gs._calcClass = UnitaryGateMatrixCalc
gs._calcClass = UnitaryGateMapCalc
gs['Gt'] = pygsti.obj.StaticGate(Ut)
gs['Gx'] = pygsti.obj.StaticGate(Ux)
gs['rho0'] = pygsti.obj.StaticSPAMVec( v0 )
gs['M0'] = pygsti.obj.UnconstrainedPOVM({'0': pygsti.obj.StaticSPAMVec( v0 ), '1': pygsti.obj.StaticSPAMVec( v1 )})
             

In [15]:
Ux

array([[0.        +0.70710678j, 0.70710678+0.        j],
       [0.70710678+0.        j, 0.        +0.70710678j]])

In [16]:
#print(gs.product(()))
#print()
print(gs.probs(()))

OutcomeLabelDict([(('1',), 0.0), (('0',), 1.0)])


In [17]:
#print(gs.product(('Gx',)))
#print()
print(gs.probs(('Gx',)))

OutcomeLabelDict([(('1',), 0.4999999999999998), (('0',), 0.5000000000000002)])


In [18]:
#print(gs.product(('Gt',)))
#print()
print(gs.probs(('Gt',)))

OutcomeLabelDict([(('1',), 0.0), (('0',), 1.0)])


In [19]:
#print(gs.product(('Gt','Gx')))
#print()
print(gs.probs(('Gt','Gx')))

OutcomeLabelDict([(('1',), 0.4999999999999998), (('0',), 0.5000000000000002)])


In [20]:
#print(gs.product(('Gx','Gx')))
#print()
print(gs.probs(('Gx','Gx')))

OutcomeLabelDict([(('1',), 0.9999999999999998), (('0',), 1.1093356479670479e-31)])


## Simulation of a "standard" n-qubit gate set
Based off of some of the initial cells in other test notebook.

In [21]:
import pygsti
import numpy as np
# A list of standard hard-coded gate labels
#gllist = ['I', 'H','P','CNOT']
gllist = ['Gi','Gh','Gp','Gcnot','Gt'] # note: must include Gt here in addition to in 'unitaries' below

# A dictionary of unitaries that do not need to be already known to the code
#unitaries={'T' : np.array([[1.,0.],[0.,np.exp(np.pi*1j/4)]])}
unitaries={'Gt' : np.array([[1.,0.],[0.,np.exp(np.pi*1j/4)]])}

# The number of qubits
n = 10

# The availiability of gates that are not available to all qubit / qubit pairs
availability = {}

# Let's make a the CNOT gate be connected in a directed ring.
# a = np.zeros((n,n),int)
# for i in range(0,n-1):
#     a[i,i+1] = 1
# a[n-1,0] = 1
a = [(i,i+1) for i in range(0,n-1)] + [(n-1,0)]

#availability={'CNOT':a}
availability={'Gcnot':a}

BGS = pygsti.construction.build_nqubit_standard_gateset(
    nQubits=n, gate_names=gllist, nonstd_gate_unitaries=unitaries,
    availability=availability, parameterization='static', sim_type="dmmap")

In [22]:
#print(BGS.preps['rho0'])
#print(list(map(str,BGS.gates.keys())))
#print(BGS)

In [23]:
# test circuit simulation
circuit = pygsti.obj.Circuit(gatestring=( ('Gh',1),('Gp',2),('Gcnot',1,2),('Gt',3) ), num_lines=n)
%time print(BGS.probs(circuit))
# ~1 min for 10Q

OutcomeLabelDict([(('0000000000',), 0.5000000000000001), (('0000000001',), 0.5000000000000001), (('0000000010',), 0.5000000000000001), (('0000000011',), 0.5000000000000001), (('0000000100',), 0.5000000000000001), (('0000000101',), 0.5000000000000001), (('0000000110',), 0.5000000000000001), (('0000000111',), 0.5000000000000001), (('0000001000',), 0.5000000000000001), (('0000001001',), 0.5000000000000001), (('0000001010',), 0.5000000000000001), (('0000001011',), 0.5000000000000001), (('0000001100',), 0.5000000000000001), (('0000001101',), 0.5000000000000001), (('0000001110',), 0.5000000000000001), (('0000001111',), 0.5000000000000001), (('0000010000',), 0.5000000000000001), (('0000010001',), 0.5000000000000001), (('0000010010',), 0.5000000000000001), (('0000010011',), 0.5000000000000001), (('0000010100',), 0.5000000000000001), (('0000010101',), 0.5000000000000001), (('0000010110',), 0.5000000000000001), (('0000010111',), 0.5000000000000001), (('0000011000',), 0.5000000000000001), (('0000

## Processor Specs

In [24]:
gllist = ['Gi', 'Gh','Gp','Gcnot','Gt']
ps = pygsti.obj.ProcessorSpec(nQubits=n, gate_names=gllist,
                              nonstd_gate_unitaries=unitaries,
                              availability=availability, verbosity=0)
  # automatically creates 'clifford' and 'target' models and std compilations
    
# To manually add another model, for example one w/density matrix simulation:
ps.models['dmsim'] = pygsti.construction.build_nqubit_standard_gateset(
    n, gllist, unitaries, availability, parameterization="static", sim_type="dmmap")
# OR, more conveniently
ps.add_std_model('dmsim', parameterization="static", sim_type="dmmap")

/Users/enielse/research/pyGSTi/packages/pygsti/construction/gatesetconstruction.py:1142: UserWarning:

Failed to create clifford gate Gt. Dropping it.



In [25]:
#Check out clifford gate on subsets of qubits (used in RB calcs)
#ps.clifford_gates_on_qubits

In [26]:
#ps.add_std_model('test', parameterization="static", sim_type="svmap")
gs = ps.models['target'].copy()
#print(list(gs.gates.keys()))
print(gs[('Gcnot',1,2)])
gs[('Gcnot',1,2)] = np.identity(4,'complex')
#NOTE: pygsti.tools.unitary_to_process_mx # unitary to paulimx
print(gs[('Gcnot',1,2)])

Embedded gate map with full dimension 1048576 and state space (0, 1, 2, 3, 4, 5, 6, 7, 8, 9)
 that embeds the following 4-dimensional gate into acting on the (1, 2) space
Static gate with shape (4, 4)
 1.00   0   0   0
   0 1.00   0   0
   0   0   0 1.00
   0   0 1.00   0

Embedded gate map with full dimension 1024 and state space (0, 1, 2, 3, 4, 5, 6, 7, 8, 9)
 that embeds the following 4-dimensional gate into acting on the (1, 2) space
Static gate with shape (4, 4)
 1.00   0   0   0
   0 1.00   0   0
   0   0 1.00   0
   0   0   0 1.00



In [27]:
#See where symplectic reps are stored (could also extract via gs.get_clifford_symplectic_reps() )
gs = ps.models['clifford']
print(gs[('Gcnot',0,1)].embedded_gate.smatrix) # each gate has it's own symplectic rep
assert(gs[('Gcnot',1,2)].embedded_gate is gs[('Gcnot',0,1)].embedded_gate) # different CNOT's share same "root"

[[1 0 0 0]
 [1 1 0 0]
 [0 0 1 1]
 [0 0 0 1]]


In [28]:
# Let's pick a random Clifford, and the inverse of that Clifford, and create an identity circuit
s, p = pygsti.tools.random_clifford(n)
sin, pin = pygsti.tools.inverse_clifford(s,p)

%time c = pygsti.alg.compile_clifford(s, p, pspec=ps)
%time c.append_circuit(pygsti.alg.compile_clifford(sin, pin, pspec=ps))
# ~3.5min each for 8Q, ~14min each for 10Q

print(len(c)) # 1389 for 8Q, 3057 for 10Q
print(c.depth()) # 1529 for 10Q

CPU times: user 3.55 s, sys: 13.3 ms, total: 3.57 s
Wall time: 3.58 s
CPU times: user 3.77 s, sys: 11.1 ms, total: 3.78 s
Wall time: 3.79 s
3016
1536


In [29]:
# As we can see, the output is (0,0,0,0) with probability 1. (when no input is given to the simulators, the input
# is taken to be (0,0,0,0)).
#%load_ext line_profiler

%time simout = c.simulate(ps.models['target'])
# ~ 3.5s for 8Q, 47s fo 10Q

print(simout['0'*n])
x = 0
for key in list(simout.keys()):
    x += simout[key]
x = x - simout['0'*n]
print(x)

CPU times: user 48.6 s, sys: 221 ms, total: 48.8 s
Wall time: 49 s
0.999999999999694
1022.9999999996734
